In [1]:
import torch
import torch.nn as nn
import numpy as np
import torch.nn.functional as F
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

import torchvision
from torchvision import datasets, transforms

from matplotlib import pyplot as plt
from IPython.display import clear_output

import os
from PIL import Image

In [2]:
import time

In [3]:
batch_size = 16
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [4]:
folder_path = 'Img_for_video'

# Получите список файлов в папке
image_files = [f for f in os.listdir(folder_path) if f.endswith('.png')]

# Инициализируйте список для хранения массивов изображений
image_arrays = []
i = 0
# Загрузите каждое изображение, преобразуйте его в массив и добавьте в список
for image_file in image_files:
    image_path = os.path.join(folder_path, image_file)
    with Image.open(image_path) as img:
        image_array = np.array(img)
        image_arrays.append(image_array)
    i += 1
    if i % 1000 == 0:
        print(f'Загрузилось {i} картинок')
# Преобразуйте список массивов в один numpy массив
orig_array = np.array(image_arrays)

print(orig_array.shape)

Загрузилось 1000 картинок
Загрузилось 2000 картинок
(2177, 420, 420, 3)


In [5]:
class OrigDataset(Dataset):
    def __init__(self, x):
        self.orig_transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Resize((420, 420))
        ])
        self.x = x
         
    def __len__(self):
        return len(self.x)
        
    def __getitem__(self, idx):
        x = self.orig_transform(self.x[idx])
        return x

In [6]:
dataset =OrigDataset(orig_array)
dataloader = DataLoader(dataset, batch_size = batch_size, shuffle = False)
org_array = None

In [7]:
def rgb_to_YCC(x):
    r = x[:, 0, :, :].unsqueeze(1)
    g = x[:, 1, :, :].unsqueeze(1)
    b = x[:, 2, :, :].unsqueeze(1)
    y = 0.299 * r + 0.587 * g + 0.114 * b
    cb = (b - y) * 0.564 + 0.5
    cr = (r - y) * 0.713 + 0.5
    return y, cb, cr

def YCC_to_rgb(x):
    y = x[:, 0, :, :].unsqueeze(1)
    cb = x[:, 1, :, :].unsqueeze(1)
    cr = x[:, 2, :, :].unsqueeze(1)
    r = y + 1.402 * (cr - 0.5)
    g = y - 0.344136 * (cb - 0.5) - 0.714136 * (cr - 0.5)
    b = y + 1.772 * (cb - 0.5)
    img = torch.cat((r, g, b), dim = 1)
    return img

class DenseBlock(nn.Module):
    def __init__(self, channels, num_conv):
        super().__init__()
        self.convs = nn.ModuleList([nn.Conv2d(in_channels = channels, out_channels = channels, kernel_size = 3, padding = 1) for _ in range(num_conv - 1)])
        self.c = nn.Conv2d(channels, channels, 3, padding = 1)
        
    def forward(self, x):
        x_prev = []
        #print(f'x dense shape:{x.shape}')
        for conv in self.convs:
            x_prev.append(x)
            #print(f'x dense shape:{x.shape}')
            x = F.leaky_relu(conv(x))
            x = x + sum(x_prev)
        x = self.c(x)
        return x


class RRDB(nn.Module):
    def __init__(self, beta = 0.2, num_blocks = 5):
        super().__init__()
        self.beta = beta
        self.blocks = nn.ModuleList([DenseBlock(64, 5) for _ in range(num_blocks)])
        
    def forward(self, x):
        x0 = x
        for block in self.blocks:
            x1 = x
            x = block(x)
            x = x1 + self.beta * x
        x = self.beta * x + x0
        return x

class Encoder(nn.Module):
    def __init__(self, in_channels, channels):
        super().__init__()
        self.c1 = nn.Conv2d(in_channels = in_channels, out_channels = channels, kernel_size = 1)
        self.c2 = nn.Conv2d(in_channels = channels, out_channels = channels, kernel_size = 3, padding = 1)
        self.c3 = nn.Conv2d(in_channels = channels, out_channels = channels, kernel_size = 3, padding = 1)
    def forward(self, x):
        x = self.c1(x)
        x = F.leaky_relu(self.c2(x))
        x = F.leaky_relu(self.c3(x))
        return x
        
class Decoder(nn.Module):
    def __init__(self, channels, out_channels):
        super().__init__()
        self.c1 = nn.Conv2d(in_channels = channels, out_channels = channels, kernel_size = 3, padding = 1)
        self.c2 = nn.Conv2d(in_channels = channels, out_channels = channels, kernel_size = 3, padding = 1)
        self.c3 = nn.Conv2d(in_channels = channels, out_channels = out_channels, kernel_size = 1, padding = 0)
    def forward(self, x):
        x = F.leaky_relu(self.c1(x))
        x = F.leaky_relu(self.c2(x))
        x = F.tanh(self.c3(x))
        return x

class Luma(nn.Module):
    def __init__(self):
        super().__init__()
        self.enc = Encoder(in_channels = 1, channels = 64)
        self.rdrb = RRDB(num_blocks = 5)
        self.dec = Decoder(channels = 64, out_channels = 1)
    def forward(self, x):
        x = self.enc(x)
        x = self.rdrb(x)
        x = self.dec(x)
        return x

class Chroma(nn.Module):
    def __init__(self):
        super().__init__()
        self.enc = Encoder(in_channels = 3, channels = 64)
        self.rdrb = RRDB(num_blocks = 3)
        self.dec = Decoder(channels = 64, out_channels = 2)
    def forward(self, x):
        x = self.enc(x)
        x = self.rdrb(x)
        x = self.dec(x)
        return x
        
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.luma = Luma()
        self.chroma = Chroma()
    def forward(self, x):
        y, cb, cr = rgb_to_YCC(x)
        #print(f'y shape:{y.shape}')
        y = self.luma(y)
        #print(f'yl shape:{y.shape}')
        y1 = y
        ycbcr = torch.cat((y, cb, cr), dim = 1)
        cbcr = self.chroma(ycbcr)
        x = torch.cat((y, cbcr), dim = 1)
        x = YCC_to_rgb(x)
        return x

In [8]:
md = Model().to(device)
md.load_state_dict(torch.load('model.pth'))
md.eval()

Model(
  (luma): Luma(
    (enc): Encoder(
      (c1): Conv2d(1, 64, kernel_size=(1, 1), stride=(1, 1))
      (c2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (c3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    )
    (rdrb): RRDB(
      (blocks): ModuleList(
        (0-4): 5 x DenseBlock(
          (convs): ModuleList(
            (0-3): 4 x Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          )
          (c): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        )
      )
    )
    (dec): Decoder(
      (c1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (c2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (c3): Conv2d(64, 1, kernel_size=(1, 1), stride=(1, 1))
    )
  )
  (chroma): Chroma(
    (enc): Encoder(
      (c1): Conv2d(3, 64, kernel_size=(1, 1), stride=(1, 1))
      (c2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1

In [9]:
model_parameters = filter(lambda p: p.requires_grad, md.parameters())
params = sum([np.prod(p.size()) for p in model_parameters])

In [11]:
print(params)

223996675


In [9]:
def Images_Process(batch, model):
    batch = batch.to(device)
    with torch.no_grad():
        y = model(batch)
    
    y = y.detach().to('cpu').numpy() #возможно не надо detach
    y = y.transpose((0, 2, 3, 1))
    y[y < 0] = 0
    y = (y * 255).astype(np.uint8)

    return y

In [10]:
start_time = time.time()
for i, batch in enumerate(dataloader):
    images = Images_Process(batch, md)
    for j, img_numpy in enumerate(images):
        img = Image.fromarray(img_numpy)
        img.save(os.path.join('Img_Video',f'{i * batch_size + j + 1}.png'))
    if i % 2 == 0:
        print(f'Прошло времени: {time.time() - start_time}')
        print(f'Создалось изображений: {(i + 1) * batch_size}')
        print(f'Изображений в секунду: {((i + 1) * batch_size)/(time.time() - start_time)}')

Прошло времени: 6.41573691368103
Создалось изображений: 32
Изображений в секунду: 4.987735692802901
Прошло времени: 18.630280017852783
Создалось изображений: 96
Изображений в секунду: 5.152901615435
Прошло времени: 30.96912455558777
Создалось изображений: 160
Изображений в секунду: 5.16643600024306
Прошло времени: 42.89606523513794
Создалось изображений: 224
Изображений в секунду: 5.221924173513993
Прошло времени: 54.98051595687866
Создалось изображений: 288
Изображений в секунду: 5.2382193034688695
Прошло времени: 67.48354029655457
Создалось изображений: 352
Изображений в секунду: 5.216086744310474
Прошло времени: 80.29606556892395
Создалось изображений: 416
Изображений в секунду: 5.180826694963241
Прошло времени: 92.717529296875
Создалось изображений: 480
Изображений в секунду: 5.17701456930624
Прошло времени: 104.14411497116089
Создалось изображений: 544
Изображений в секунду: 5.223530874986474
Прошло времени: 116.05178546905518
Создалось изображений: 608
Изображений в секунду: 5.23